<a href="https://colab.research.google.com/github/fportoDexl/cursoBigData/blob/main/K_means_MR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## K-Means Versão spark
  Neste exemplo consideramos que geramos 1000 pontos randomicos, no intervalo
  [1,5000] e queremos clusterizar em k grupos.

Iniciamos instalando o pyspark

In [1]:
pip install pyspark


     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 53.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=7f4bfcc564fb801a2376e03ca2440ebf0ce6292f392e270be865fb8779237eb1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


## Importamos pyspark - API python para Spark
##            radom - para geração pesudo-aleatório de inteiros

In [9]:
import pyspark as ps
import random

# Importamos o SparkContext para definição do espaço RDD
Deve existir apenas um SparkContext por job spark

In [10]:
from pyspark.context import SparkContext
sc= SparkContext(appName="K-means", master="local[4]")

ValueError: ignored

## Iniciamos um acumulador - Fica armazenado no Driver e pode ser acumulado 
## a partir dos nós trabalhadores

In [11]:
acum=sc.accumulator(0)

## Criamos uma classe para iniciar as duas listas:

*   centroids - conjunto de centroids dos clustes. Inicialmente vazia
*   points - lista de pontos a clusterizar


  - 

In [12]:
class K_means_exemplo_MR:
  def __init__(self, k):
    self.centroids=list(random.sample(range(1,1000),k))
    self.points=sc.parallelize(random.sample(range(1,5000),1000),2)
  


## Função findCluster
  input= p-> ponto sendo avaliado; centroids-> lista de centroids
  Objetivo:
    c=varrer a lista e encontrar o centroid mais proximo ao ponto "p"

In [13]:
def findCluster(p,centroids):
    min=999999
    for c in centroids:
      newdist=abs(p-c)
      if  newdist < min:
        centroid=c
        min = newdist
    return [centroid,p]

## Função K_Means - função principal
  obtém a lista de centroids  randômicos - iniciais (centroids_updt)
  inicializa a lista de centroids
  realiza um loop até que o processo convirja e centroids == centroids_updt
     a função map chama a findCluster-> forma o par (centroid, ponto)
     utiliza a função groupByKey para gerar (K,[V])
     calcula a media com a soma/|V| dos valores em V
     transforma a tupla em  RDD com centroids
     incrementa no acumulador o numero de iterações

In [14]:
def k_means(k):

  k_means = K_means_exemplo_MR(k)
  centroids = list([])
  centroids_updt = list(k_means.centroids)
  print("centroids"+str((centroids)))
  print("centroid_upd"+str(centroids_updt))
  points=k_means.points
  print(str(points.count()))
  iter=0
  while (sorted(centroids) != sorted(centroids_updt)):
    centroids=centroids_updt
    keysValues=points.map(lambda j:findCluster(j,centroids))
    reducedKeys=keysValues.groupByKey().mapValues(lambda x: sum(x) / len(x))
    newKeys=reducedKeys.map(lambda x:x[1])
    centroids_updt=newKeys.collect()
    acum.add(1)

  return centroids_updt

## Invoca o process

In [15]:
k=3
groups = k_means(k)
print(str(groups)+ "Number of iterations: "+str(acum))

centroids[]
centroid_upd[21, 330, 519]
1000
[4072.2561307901906, 778.3677419354839, 2440.4210526315787]Number of iterations: 17
